# Clotho Dataset Preparation

This notebook will:
1. Download Clotho dataset using `aac_datasets`
2. Extract mel spectrograms from audio files
3. Create caption JSON files
4. Build vocabulary

**Estimated time**: 30-60 minutes (first run only, includes download)

## Step 1: Setup and Imports

In [1]:
import os
os.environ['MKL_THREADING_LAYER'] = 'GNU'  # Fix threading issue

import sys
from pathlib import Path

# Add parent directory to path
project_root = Path('..').absolute()
sys.path.insert(0, str(project_root))

import numpy as np
import json
import librosa
from tqdm.notebook import tqdm
import time

print("✓ Imports successful!")
print(f"Project root: {project_root}")

✓ Imports successful!
Project root: /home/harsha/Documents/ncsu/sem 1/ALDA/Project/cross-model-audio-text-captioning/notebooks/..


## Step 2: Check/Install aac_datasets

In [2]:
try:
    from aac_datasets import Clotho
    print("✓ aac_datasets is installed")
except ImportError:
    print("Installing aac_datasets...")
    !pip install aac-datasets
    from aac_datasets import Clotho
    print("✓ aac_datasets installed successfully")

✓ aac_datasets is installed


## Step 3: Setup Paths

In [3]:
# Create directories
clotho_root = project_root / 'clotho_data'
mel_dir = project_root / 'features' / 'mel'
mel_eval_dir = project_root / 'features' / 'mel_eval'
data_dir = project_root / 'data'

# Create all directories
for directory in [clotho_root, mel_dir, mel_eval_dir, data_dir]:
    directory.mkdir(parents=True, exist_ok=True)
    print(f"✓ Created: {directory.relative_to(project_root)}")

print("\n✓ All directories ready!")

✓ Created: clotho_data
✓ Created: features/mel
✓ Created: features/mel_eval
✓ Created: data

✓ All directories ready!


## Step 4: Download Clotho Dataset

**This will take 10-30 minutes on first run** (downloads ~2-3 GB)

Subsequent runs will be fast as data is cached.

In [4]:
print("="*80)
print("DOWNLOADING CLOTHO DATASET")
print("="*80)
print("\n⏰ This may take 10-30 minutes on first run...\n")

start_time = time.time()

try:
    # Load train split (development)
    print("📥 Loading training data (development split)...")
    train_dataset = Clotho(
        root=str(clotho_root),
        subset='dev',
        download=True
    )
    print(f"✓ Training samples: {len(train_dataset)}")
    
    # Load validation split
    print("\n📥 Loading validation data...")
    val_dataset = Clotho(
        root=str(clotho_root),
        subset='val',
        download=True
    )
    print(f"✓ Validation samples: {len(val_dataset)}")
    
    # Load evaluation split
    print("\n📥 Loading evaluation data...")
    eval_dataset = Clotho(
        root=str(clotho_root),
        subset='eval',
        download=True
    )
    print(f"✓ Evaluation samples: {len(eval_dataset)}")
    
    elapsed = time.time() - start_time
    print(f"\n✓ Dataset loaded in {elapsed/60:.1f} minutes")
    
except Exception as e:
    print(f"\n❌ Error: {e}")
    print("\nTroubleshooting:")
    print("1. Check internet connection")
    print("2. Make sure you have ~5 GB free space")
    print("3. Try running again (download will resume)")
    raise

DOWNLOADING CLOTHO DATASET

⏰ This may take 10-30 minutes on first run...

📥 Loading training data (development split)...
✓ Training samples: 3839

📥 Loading validation data...
✓ Validation samples: 1045

📥 Loading evaluation data...
✓ Evaluation samples: 1045

✓ Dataset loaded in 58.8 minutes


## Step 5: Inspect Sample Data

In [12]:
# Look at a sample
sample = train_dataset[0]

print("Sample data structure:")
print(f"  Keys: {sample.keys()}")
print(f"\n  Audio shape: {sample['audio'].shape}")
print(f"  Sample rate: {sample['sr']} Hz")
print(f"  Duration: {len(sample['audio'])/sample['sr']:.2f} seconds")
print(f"\n  Captions ({len(sample['captions'])}):")
for i, caption in enumerate(sample['captions'], 1):
    print(f"    {i}. {caption}")

Sample data structure:
  Keys: dict_keys(['dataset', 'index', 'subset', 'keywords', 'sound_link', 'audio', 'start_end_samples', 'duration', 'manufacturer', 'fname', 'sr', 'license', 'sound_id', 'captions'])

  Audio shape: torch.Size([1, 1153825])
  Sample rate: 44100 Hz
  Duration: 0.00 seconds

  Captions (5):
    1. A muddled noise of broken channel of the TV
    2. A television blares the rhythm of a static TV.
    3. Loud television static dips in and out of focus
    4. The loud buzz of static constantly changes pitch and volume.
    5. heavy static and the beginnings of a signal on a transistor radio


## Step 6: Extract Mel Spectrograms

This will process all audio files and extract mel spectrograms.

**Estimated time**: 15-30 minutes for all splits

In [13]:
def extract_mel_spectrogram(audio, sr=32000, n_mels=64, n_fft=2048, hop_length=512):
    """
    Extract mel spectrogram from audio
    """
    # Extract mel spectrogram
    mel = librosa.feature.melspectrogram(
        y=audio,
        sr=sr,
        n_mels=n_mels,
        n_fft=n_fft,
        hop_length=hop_length
    )
    
    # Convert to log scale (dB)
    mel = librosa.power_to_db(mel, ref=np.max)
    
    # Add channel dimension
    mel = mel[np.newaxis, ...]  # (1, n_mels, time_steps)
    
    return mel

print("✓ Mel extraction function ready")

✓ Mel extraction function ready


In [16]:
def process_split(dataset, output_mel_dir, output_json_path, split_name):
    """
    Process one split: extract mels and save captions (FIXED VERSION)
    """
    print(f"\n{'='*80}")
    print(f"Processing: {split_name.upper()}")
    print(f"{'='*80}")
    
    data = []
    errors = 0
    
    for idx in tqdm(range(len(dataset)), desc=f"Processing {split_name}"):
        try:
            item = dataset[idx]
            
            # Get data
            audio = item['audio']
            sr = item['sr']
            captions = item['captions']
            fname = item.get('fname', f"{split_name}_{idx:05d}")
            
            # Convert audio to numpy array if it's a tensor
            if not isinstance(audio, np.ndarray):
                audio = np.array(audio)
            
            # Ensure audio is 1D
            if audio.ndim > 1:
                audio = audio.squeeze()
            
            # Convert to float32 if needed
            if audio.dtype != np.float32:
                audio = audio.astype(np.float32)
            
            # Resample if needed
            if sr != 32000:
                audio = librosa.resample(audio, orig_sr=sr, target_sr=32000)
            
            # Extract mel
            mel = extract_mel_spectrogram(audio, sr=32000)
            
            # Save mel
            mel_path = output_mel_dir / f"{fname}.npy"
            np.save(mel_path, mel)
            
            # Add to captions
            data.append({
                'fname': fname,
                'captions': captions if isinstance(captions, list) else [captions]
            })
            
        except Exception as e:
            errors += 1
            if errors <= 3:  # Show first 3 errors
                print(f"\n⚠ Error at index {idx}: {e}")
                import traceback
                traceback.print_exc()
    
    # Save caption JSON
    with open(output_json_path, 'w') as f:
        json.dump(data, f, indent=2)
    
    print(f"\n✓ Processed: {len(data)} samples")
    if errors > 0:
        print(f"⚠ Errors: {errors} samples")
    print(f"✓ Saved mels to: {output_mel_dir.relative_to(project_root)}")
    print(f"✓ Saved captions to: {output_json_path.relative_to(project_root)}")
    
    return len(data)

print("✓ FIXED processing function ready - Now re-run the training data cell!")

✓ FIXED processing function ready - Now re-run the training data cell!


### Process Training Data

In [17]:
train_count = process_split(
    dataset=train_dataset,
    output_mel_dir=mel_dir,
    output_json_path=data_dir / 'train_captions.json',
    split_name='train'
)


Processing: TRAIN


Processing train:   0%|          | 0/3839 [00:00<?, ?it/s]

/tmp/ipykernel_40177/4095576106.py:24: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  audio = np.array(audio)
/tmp/ipykernel_40177/4095576106.py:24: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  audio = np.array(audio)



✓ Processed: 3839 samples
✓ Saved mels to: features/mel
✓ Saved captions to: data/train_captions.json


### Process Validation Data

In [18]:
val_count = process_split(
    dataset=val_dataset,
    output_mel_dir=mel_dir,
    output_json_path=data_dir / 'val_captions.json',
    split_name='val'
)


Processing: VAL


Processing val:   0%|          | 0/1045 [00:00<?, ?it/s]

/tmp/ipykernel_40177/4095576106.py:24: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  audio = np.array(audio)



✓ Processed: 1045 samples
✓ Saved mels to: features/mel
✓ Saved captions to: data/val_captions.json


### Process Evaluation Data

In [19]:
eval_count = process_split(
    dataset=eval_dataset,
    output_mel_dir=mel_eval_dir,
    output_json_path=data_dir / 'eval_captions.json',
    split_name='eval'
)


Processing: EVAL


Processing eval:   0%|          | 0/1045 [00:00<?, ?it/s]

/tmp/ipykernel_40177/4095576106.py:24: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  audio = np.array(audio)



✓ Processed: 1045 samples
✓ Saved mels to: features/mel_eval
✓ Saved captions to: data/eval_captions.json


## Step 7: Dataset Statistics

In [20]:
print("\n" + "="*80)
print("DATASET STATISTICS")
print("="*80)

total = train_count + val_count + eval_count
print(f"\nTotal samples processed: {total}")
print(f"  - Training: {train_count}")
print(f"  - Validation: {val_count}")
print(f"  - Evaluation: {eval_count}")

# Analyze captions
for split_name, json_path in [
    ('Training', data_dir / 'train_captions.json'),
    ('Validation', data_dir / 'val_captions.json'),
    ('Evaluation', data_dir / 'eval_captions.json')
]:
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    caption_lengths = [len(cap.split()) for item in data for cap in item['captions']]
    
    print(f"\n{split_name}:")
    print(f"  - Total captions: {len(caption_lengths)}")
    print(f"  - Avg caption length: {np.mean(caption_lengths):.1f} words")
    print(f"  - Min/Max length: {np.min(caption_lengths)}/{np.max(caption_lengths)} words")

print("\n" + "="*80)


DATASET STATISTICS

Total samples processed: 5929
  - Training: 3839
  - Validation: 1045
  - Evaluation: 1045

Training:
  - Total captions: 19195
  - Avg caption length: 11.3 words
  - Min/Max length: 8/21 words

Validation:
  - Total captions: 5225
  - Avg caption length: 11.4 words
  - Min/Max length: 8/21 words

Evaluation:
  - Total captions: 5225
  - Avg caption length: 11.3 words
  - Min/Max length: 8/21 words



## Step 8: Build Vocabulary

In [21]:
from src.utils import build_vocab, save_vocab

print("="*80)
print("BUILDING VOCABULARY")
print("="*80)

vocab, word_freq = build_vocab(
    captions_files=[
        str(data_dir / 'train_captions.json'),
        str(data_dir / 'val_captions.json')
    ],
    min_word_freq=2
)

# Save vocabulary
vocab_path = project_root / 'vocab.json'
save_vocab(vocab, str(vocab_path))

print(f"\n✓ Vocabulary size: {len(vocab)}")
print(f"\nMost common words:")
for word, count in word_freq.most_common(20):
    print(f"  {word:<15} {count:>5}")

BUILDING VOCABULARY
Vocabulary size: 5986
Total unique words: 7819
Words with freq >= 2: 5982
Vocabulary saved to /home/harsha/Documents/ncsu/sem 1/ALDA/Project/cross-model-audio-text-captioning/notebooks/../vocab.json

✓ Vocabulary size: 5986

Most common words:
  a               24739
  the             13138
  and             12179
  is               9145
  in               7299
  of               4807
  are              4059
  as               3653
  with             3002
  while            2966
  on               2949
  then             2360
  water            2310
  birds            2259
  an               2229
  people           2159
  to               2130
  someone          1897
  by               1842
  person           1819


## Step 9: Summary

In [22]:
print("\n" + "="*80)
print("✓ PREPARATION COMPLETE!")
print("="*80)

print("\nFiles created:")
print(f"  ✓ {mel_dir.relative_to(project_root)}/ - {train_count + val_count} mel spectrograms")
print(f"  ✓ {mel_eval_dir.relative_to(project_root)}/ - {eval_count} mel spectrograms")
print(f"  ✓ {data_dir.relative_to(project_root)}/train_captions.json")
print(f"  ✓ {data_dir.relative_to(project_root)}/val_captions.json")
print(f"  ✓ {data_dir.relative_to(project_root)}/eval_captions.json")
print(f"  ✓ vocab.json")

print("\n" + "="*80)
print("NEXT STEPS")
print("="*80)
print("\n1. Train baseline model:")
print("   python scripts/train_baseline.py")
print("\n2. Or continue in notebooks:")
print("   Open: notebooks/02_train_baseline.ipynb")
print("\n" + "="*80)


✓ PREPARATION COMPLETE!

Files created:
  ✓ features/mel/ - 4884 mel spectrograms
  ✓ features/mel_eval/ - 1045 mel spectrograms
  ✓ data/train_captions.json
  ✓ data/val_captions.json
  ✓ data/eval_captions.json
  ✓ vocab.json

NEXT STEPS

1. Train baseline model:
   python scripts/train_baseline.py

2. Or continue in notebooks:
   Open: notebooks/02_train_baseline.ipynb

